In [ ]:
#| default_exp session_1

# 1. Session - MalariaGEN

> The first session about the notebook is about exploring `https://www.malariagen.net/`. 

MalariaGen.net is an online database that provides comprehensive genomic data on Plasmodium falciparum, the parasite that causes the most severe form of malaria in humans. The database was established by the Malaria Genomic Epidemiology Network (MalariaGEN), which is a global collaboration of researchers working towards understanding the genetic and molecular basis of malaria susceptibility, drug resistance, and transmission.

The database includes information on the genetic diversity of the parasite, including data on single nucleotide polymorphisms (SNPs) and copy number variations (CNVs). It also provides information on the distribution of drug resistance mutations, which is critical for developing effective malaria treatments.

In addition to genomic data, MalariaGen.net also provides access to other types of data, including epidemiological and clinical data. These data are integrated with the genomic data to provide a comprehensive view of the malaria landscape, including information on the distribution of different malaria parasite strains and their associated clinical outcomes.

Overall, MalariaGen.net is an essential resource for researchers and public health officials working towards the goal of eliminating malaria. By providing open access to comprehensive genomic and epidemiological data, MalariaGen.net is helping to drive advances in our understanding of the biology of the malaria parasite and to inform the development of new and effective malaria control strategies.



